## Kavir_RAG
Retrieval-Augmented Generation
<div style="direction: rtl; white-space: pre-wrap; line-height: 1.5;">
Retrieval = بازیابی (درست مثل سرچ کردن و پیدا کردن یک متن مرتبط از میان کلی سند)
Augmented = تقویت‌شده / غنی‌شده (یعنی اطلاعات جدیدی به چیزی اضافه شده)
Generation = تولید متن (کاری که مدل زبانی انجام می‌ده: جمله‌سازی، پاسخ‌دهی)
</div>

<div style="direction: rtl;">


# تعریف پروژه (گام ۰: صورت‌مسئله و برنامهٔ کار)

## نام‌ها

* **نام پروژه (repo):** `Kavir-RAG`  *(کویر: محلی، مستقل، خلوت)*
* **نام نوت‌بوک اصلی:** `Kavir_RAG_MVP.ipynb`

## هدف فنی

یک **سرور RAG کاملاً لوکال و آفلاین** برای تحلیل اسناد چندفرمت (PDF, DOCX, XLSX/CSV, TXT, code snippets, IPYNB) با:

* ایندکس‌سازی (FAISS/Chroma) + امبدینگ چندزبانه (fa/en)
* بازیابی معنایی + (اختیاری) ریرنکر
* پاسخ LLM لوکال با **ارجاع به منبع**
* API مبتنی بر **FastAPI** (endpoints: `/upload`, `/query`, `/chat`, `/reindex`) با **توکن ساده**
* اجرا روی i5 / 8GB RAM (CPU-only؛ 16GB روان‌تر)

## پیکرهٔ فنی پیشنهادی

* **LLM**: Qwen2-7B-Instruct (Q4\_0 via Ollama/llama.cpp) – کم‌مصرف و چندزبانه
* **Embedding**: `intfloat/multilingual-e5-small` (CPU-friendly)
* **Vector DB**: FAISS (IndexFlatIP + نُرمال‌سازی → cosine)
* **Re-rank (اختیاری)**: `bge-reranker-base` (CPU)
* **Parserها**: PyMuPDF (PDF)، python-docx، pandas (CSV/XLSX)، nbformat (ipynb)، Tesseract (OCR در صورت نیاز)
* **API**: FastAPI + Uvicorn
* **UI سبک (اختیاری)**: Streamlit (یا React ساده)
* **امنیت LAN**: توکن ساده + (اختیاری) Nginx/Caddy با TLS داخلی


## خروجی‌های نسخهٔ پایه (MVP)

* Endpointهای `/upload`, `/query`, `/reindex` (و `/chat` سبک)
* بازیابی top-k با citation
* ذخیرهٔ ایندکس روی دیسک و بارگذاری مجدد
* اجرای کامل روی یک ماشین بدون اینترنت (بعد از دانلود اولیهٔ مدل‌ها)

## زمان‌بندی و هزینه (الگوی آپ‌ورک)

* **MVP (فقط API+RAG)**: \~ **۱ روز کاری**
  شکست به تسک‌ها: تعاریف و اسکلت (۱س) • نصب/مدل (۲س) • ingest+index (۲–۳س) • RAG+API (۲س) • تست سریع (۱س).
* **کامل‌تر (UI، امنیت LAN، بکاپ، ریرنکر، هیبرید سرچ)**: **۲–۴ روز کاری**
* **تخمین هزینه** (فرضِ فریلنسری استاندارد):

  * MVP: **۲۰۰–۳۰۰ دلار**
  * کامل‌تر: **۵۰۰–۸۰۰ دلار**
* **آپشن توسعهٔ حرفه‌ای (۳–۷ روز اضافه، +۷۰۰ تا +۱۵۰۰ دلار بسته به دامنه):**

  * هیبرید سرچ (BM25 + dense) و تیونینگ ریتریور
  * ریرنکر پیش‌فرض + A/B ارزیابی پاسخ‌ها
  * Session memory برای چت چندمرحله‌ای
  * Docker Compose + Nginx/Caddy + TLS داخلی
  * Role-based tokens، لاگ و Auditing
  * Sandbox خواندن امنِ ipynb (بدون اجرای کُد کاربر)
  * تست‌های واحد/تکاملی + بنچمارک کوچک کیفیت RAG
  * اسکریپت پشتیبان‌گیری ایندکس + بازیابی

## ریسک‌ها و قیود

* 8GB RAM کفِ قابل اجراست؛ زمان ساخت امبدینگ اولیه کندتر است.
* فایل‌های تصویریِ فرمول نیازمند OCR هستند و ممکن است کیفیت استخراج متغیر باشد.
* بارِ نخست نیاز به دانلود مدل‌ها دارد؛ سپس کاملاً آفلاین.

---

# گام بعدی (مرحله ۱: اسکلت پروژه و آماده‌سازی محیط)

در این مرحله فقط **ساختار پوشه‌ها** و یک **venv سبک** ایجاد می‌کنیم و فهرست **dependencies** را می‌نویسیم (بدون نصب سنگین مدل‌ها هنوز).
طبق «پرامپتِ قدم»، **هیچ کُدی** تا وقتی تأیید ندهی نمایش نمی‌دهم.


</div>

In [1]:
'''Kavir-RAG/
  data/
    raw/           # raw uploaded files
    ocr/           # OCR output (if needed)
    index/         # FAISS index stored on disk
  models/          # cached models (optional)
  app/
    api.py         # FastAPI endpoints
    rag_core.py    # retrieval + prompt composition + generation
    ingest.py      # file parsers + chunking + embeddings
    auth.py        # simple token-based authentication
    utils.py       # helper functions
  notebooks/
    Kavir_RAG_MVP.ipynb
  configs/
    settings.yaml  # configuration file
  tests/           # unit/integration tests
  README.md        # project documentation
  '''


'Kavir-RAG/\n  data/\n    raw/           # raw uploaded files\n    ocr/           # OCR output (if needed)\n    index/         # FAISS index stored on disk\n  models/          # cached models (optional)\n  app/\n    api.py         # FastAPI endpoints\n    rag_core.py    # retrieval + prompt composition + generation\n    ingest.py      # file parsers + chunking + embeddings\n    auth.py        # simple token-based authentication\n    utils.py       # helper functions\n  notebooks/\n    Kavir_RAG_MVP.ipynb\n  configs/\n    settings.yaml  # configuration file\n  tests/           # unit/integration tests\n  README.md        # project documentation\n  '

In [3]:
!touch Dockerfile
!touch requirements.txt
!mkdir -p src
!touch src/main.py